In [ ]:
import os
import dotenv
import base64
import logging
import time
import re
import pandas

import swagger_client
from swagger_client import configuration
from swagger_client.rest import ApiException
from pprint import pprint

def build_config():
    """ Build Config object from environment variables """
    config = configuration.Configuration()
    config.password = os.getenv('PASSWORD')
    del os.environ['PASSWORD']
    config.username = os.getenv('USER')
    config.host = os.getenv('INSIGHTVM_HOST')
    config.verify_ssl = False

    return config

def lookup_host(api_client, host =''):
    """ 
    Look up if the host is in InsightVM
    Returns ... something

    :param api_client: Instance of swagger_client.ApiClient

    :param host: Shortname, or subset of shortname, for the host that is being searched
    """
    api_instance = swagger_client.AssetApi(api_client)

    if os.getenv('FILTERS'):
        # TODO - Make this work if we ever want to validate more than the hostname
        # which we do already because we want to know if the IP is the same as what
        # is in InsightVM as the base image will go up and down
        for f in os.environ['FILTERS']:
            pprint(f)
    
    else:
        filters = {
            'field': 'host-name',
            'operator': 'is-like',
            'value': host,
        }

    body = swagger_client.SearchCriteria(match='all',filters=[filters])

    try:
        api_response = api_instance.find_assets(body, page=os.environ['PAGE'], size=os.environ['SIZE'])
        logging.debug(api_response.resources)
    except ApiException as e:
        print("Exception when calling AssetApi->find_assets: %s\n" % e)

    return api_response.resources

def update_asset(api_client, asset_id, ip):
    """
    Update resource with current asset ip

    :param api_client: A valid InsightVM client
    :param asset_id: the asset desired to update
    :param ip: the current IP of the asset
    """

    api_client = api_client
    asset_id = ""
    ip = os.environ['IP']


def scan_host(api_client, host =""):
    """
    Run security scan on the host

    Returns scan results
    """
    # TODO - check for a running scan
    api_instance = swagger_client.ScanApi(api_client)
    id = 9 # TODO - Get rid of this magic number, which might be needed for testing...
    body = swagger_client.AdhocScan() # AdhocScan | The details for the scan. (optional)

    try:
        api_response = api_instance.start_scan(id, body=body)
        logging.debug('Response: {}'.format(api_response))
    except ApiException as e:
        logging.error("Exception when calling ScanApi->start_scan: %s\n" % e)

    scan_id = api_response.id
    stuff = poll_scan(api_client, scan_id)
    
    return stuff

def poll_scan(api_client, id =""):
    """
    Polls the scan_id every 5 seconds until scan is complete

    Returns completed data
    """
    status = ""

    logging.info('Checking status of scan {}'.format(id))
    while status != 'finished':
        api_instance = swagger_client.ScanApi(api_client)
        id = id

        try:
            api_response = api_instance.get_scan(id)
            status = api_response.status
            logging.info('Scan {} is in {} status'.format(id, status))
            time.sleep(5)
        except ApiException as e:
            print("Exception when calling ScanApi->get_scan: %s\n" % e)

    return api_response

def validate_hostname(asset, hostname):
    """
    Validate the hostname matches what is on the asset

    :param dict asset: The assest as returned by InsightVM
    :param str hostname: The hostname you are looking to match
    """
    matching_assets = []

    for hostname in asset.host_names:
        if hostname and re.search(hostname, hostname.name):
            logging.info('Found {} in list of hostnames'.format(hostname))
            matching_assets.append(a)

    return matching_assets

def validate_host(assets, search_term, ip_to_scan):
    search_term = os.getenv('HOST')
    ip_to_scan = os.environ['IP']
    """
    Validate the asset returned is in fact the host we are looking for
    :param list asset: assets returned by the SearchAPI call
    :param str search_term: name of host used to search with
    :param str ip_to_scan: ip address of the host desiring to scan
    Returns single asset
    """
    ass = []
    for a in assets:
        # Asset hostname correct?
        if len(validate_hostname(a, search_term)) == 0:
            logging.errorexit()

        # IP matches vm to test?
        if ip_to_scan and not a.ip == ip_to_scan:
            logging.info('The address in InsightVM ({}) and the address needing scanned ({}) are not the same'.format(a.ip, ip_to_scan))
            # TODO - Feature: add_or_update_host if IP doesn't match
            # add_or_update_asset(api_client, asset, ip)

        # Scanned in the last two hours?
        for event in a.history:
            event_time = pandas.Timestamp(event._date)
            current_time = pandas.Timestamp.now('UTC')
            delta = current_time - event_time

            # TODO - Cleanup: Remove second two_hours that was used in testing
            two_hours = 7200.0
            two_hours = 30.0
            if delta.total_seconds() < two_hours and event.type != 'SCAN':
                logging.warning('Asset {} was scanned within the last 2h; passing'.format(a.host_name))
                exit()

        ass.append(a)


    if len(ass) > 1:
        logging.debug('Multiple assets were found; be more specific in your search/n{}'.format(assets))
        exit()

    asset = ass
    return asset

In [ ]:

# main
ENV_FILE = dotenv.find_dotenv()
if ENV_FILE:
    dotenv.load_dotenv(ENV_FILE, override=True)

host = os.getenv('HOST')
config = build_config()

api_client = swagger_client.ApiClient(config)

assets = lookup_host(api_client, host)
if assets is None:
    logging.error("We didn't get a response in our assets list")
    exit

if len(assets) < 1:
    logging.warning("{} was not found".format(host))
    logging.info("Adding {} to InsigthVM".format(host))
    add_host(api_client, host)


In [ ]:
    # if len(assets) >= 1:
    #     asset = validate_host(assets, host, os.environ['IP'])

    # logging.info('Found the host: {}'.format(host))
    ## TODO- Swap True to False as the desired normal state
    # if asset.assessed_for_vulnerabilities == True:
        # scan_host(api_client, host)
# 
    # if asset.vulnerabilities.total > 0:
        # logging.error('Total risk score is too high: {}'.format(asset.vulnerabilities.total))
        # exit()
# 
    # print("We Passed!")

    # for property, value in vars(api_instance).items():
    #     print(property, ":", value)
